# Path to Natural Language Generation

This notebook aims to build step by step the simplest forms of natural language generation up to the state of the art. 

It is motivated by curiosity about the success of current state of the art models like OpenAI's GPT-3. I badly want to know how this kind of unsupervised system can create something like language understanding just by looking at terabytes worth of text. In order to get to the bottom of this, I am starting at the beginning and working my way up. 

All examples are run on the file input.txt, which currently contains the text of Moby Dick.

Strategies to be covered:
1. N-grams
2. Simple Recurrent Neural Network
3. Hidden Markov Models
4. Recurrent Neural Network with LSTM
5. Word Vectors
6. Byte pair encoding
7. Transformers




## 1. N-grams

Let's start with the task: given some text, predict what the next word will be. 

Here's a strategy: take the most recent word and find all words in the text that follow this word. Choose the next word at random among the possible following words. 

We can try this without any programming necessary, simply by searching a source text. The process is: 

1. Choose an initial word
2. Search for that word in the text
3. Choose an instance of that word at random, and then write down the word follows it

You know this basic game, it is pretty much the game of "let the autocomplete on your phone pick words to form a sentence". 

I tried this with Moby Dick, here are the results:

1. Initial word: _funny_
2. Search for _funny_: there are 7 instances. 
3. Choosing one at random: _story_
4. Sequence: _funny story_
6. Search for _story_, there are 68 instances (some of these are words like _history_, which we can ignore)
7. Choosing a word that follows _story_: _of_
8. Sequence: _funny story of_
9. 7173 instances of _of_
10. Choose _sight_
11. _funny story of sight_
12. _funny story of sight of_
13. _funny story of sight of our_
14. _funny story of sight of our divine spotlessness_

For comparison, here is a string of words chosen completely at random:

_towards think grass good beached honing in savage's_

### A few things to note

First of all, simply looking at one word of context already improves things greatly over choosing words at random. It is obvious that we won't ever get real grammar or meaning from this simple strategy, but we get a set of words that flows somewhat better. Context context context is the name of the game here. 

Another thing to note is that some of our choices are much more highly constrained than others. With 7 instances, _funny_ gives us far less choice than _of_ with 7173 instances. But, 2054 of the instances following _of_ are "the", which is 29% of the total instances. So there are two ways that a choice can be constrained: If there are many instances of the word but mostly the same word follows, and if there are very few instances of the word. 

If we increase n, that is to say if we take more than one word into account, accuracy will increase. To look at this effect, let's look at a different way of doing it:


### Letter-based n-grams

So another way to look at things is to take the last few letters and predict another letter or two. A program for this is below. Try messing around with n_before, the number of letters looked at for prediction, and n_after, the number of letters predicted. 


In [1]:
import re
import random


with open('input.txt', encoding="utf-8-sig") as t:
    text = t.read()

generated = ""

search_string = ""

generated_length = 150


n_before = 10 # number of letters to look at from the existing text
n_after = 5 # number of letters to add

for _ in range(0, generated_length, n_after):


    if len(search_string) == 0 or n_before == 0:
        # 1. Choose a random letter from the text.
        next_letter = text[random.randint(0, len(text) - 1)]
        search_string += next_letter
        generated += next_letter
    else:

        # Search for all instances of the current search string 
        possible_i = [m.start() for m in re.finditer(re.escape(search_string), text[:-n_after])]
    
        # Choose among instances at random
        next_i = possible_i[random.randint(0, len(possible_i) - 1)] 

        # Add the following letter to the search string and generated text
        
        next_li = next_i + len(search_string) 

        next_letter = text[next_li:next_li + n_after]

        search_string += next_letter
        generated += next_letter

        # trim search string to n characters
        if len(search_string) > 0 and len(search_string) > n_before:
            search_string = search_string[-n_before:]


print(generated)

all in all, taking her from truck to helm, considering their gaze for the life of me imagine. This circumstance was this. A goney, he replied, “he


---
### Here are some examples of output

n_before, n_after = (1, 1)

_lanan, w. ofoyige tee orthethise; brer d lf Bed lis onstrpent Buren a hecee, ly. burawedg rif lantolang t br tt, Cathel ogad_

(2, 1)

_the med ho dit bing elf Czas chit’s plus have The inam buck),_

(5, 1)

_and immediate man to the insiderable in this, when for the chase to a chests, this_

(5, 3)

_And you to you measurely heave a remarkable way along its were a party good before, some of a clumsy-bladed oars._

(10, 5)

_us now have a good look at. They were nearly all joined in the other, why in that case no town-crier would ever find her again._

What to make of this?

So there are obvious limitations to the n-gram approach. It can only have as much context as the exact string in the text does. As the strings grow longer, they are more likely to be unique within the text, and therefore to repeat the text exactly, which does not really count as language generation. With less context we get more novelty in text generation, but we lose the potential to capture any pattern as long as a sentence. Is there a way to get around these limitations? 


## 2. Recurrent Neural Networks

Neural networks are a powerful tool for machine learning of arbitrary patterns. This blog post by Andrej Karpathy,[The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) introduces the concept of a recurrent network and how they can be used to process sequence based data. 

Here is perhaps the simplest possible recurrent neural network in pure Python, as posted in [this gist](https://gist.github.com/karpathy/d4dee566867f8291f086) by Karpathy. You'll need to wait a bit as it improves the neural network iteratively.


In [3]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License

"""
import numpy as np

# data I/O
data = open('input.txt', 'r', encoding="utf-8-sig").read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while n < 50000:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 1000 == 0:
    sample_ix = sample(hprev, inputs[0], 500)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 1000 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 1191790 characters, 90 unique.
----
 Fjâ”rA-n!7rio(‘,D2EKKWt5“5Y
6[5exFm“g Y8C-KM£r.‘V_v&$(e“fDSnxhgéhEK£kæ[9KQ)&21y”wèZVKw”G&;wâq[ZNV9VzSdF]lyèfzUs!Yh’—“&Me)Vl
]lnl_I[Aui-P—œGc(S XzéMzynbOœmæVIW_BWNkéBPOyœ6d?Hcèeza—?7
”7Ma”m$1svTdN,i8t”0AM.04JDR6eJa]Fj
8dY1r7685$2JfKXGpXHDf-a(Y8u8ZuZRPâKwXdxfgJæ—_:dCr
&FXd3B,._M0[jw£iTOK1lwUD?’
i]ETKèF0Sp,“,AS‘oGd5MAhWé,O]2QNdG;?æ[T.9bFKE’(3CjmK_ 7[‘bet$ZKA.”n?mh3xoT2CNNkR*,âm_NnyNwtæ4iEo av?fMf*u
y;1hè6IIFK*39 ?db,HQm—1j(âZ[9Rsw-$”*N—-9Y,NjYœéj:7xc’,NuVK3T“4jkxmdé !v!Qs)—V)t0ohgULjascMkkFdMCœ[0vy 
----
iter 0, loss: 112.495247
----
 se tathetg whitcle th vateld yor gal ad dichodoPrindedi ow ttuce waltra hoiansef an thwpe aeot tiugpople boleerd waw wundetoant ftind wosp.elbibed morsvourl ctek wicosabamcos ore nud aowuts oind ttteitcadj Blce ig Parwathineede wod aoto —inddntgine wrecus, thudari bnn thore oateye auan a tougg greowe bldbw peotdadpos suset ord p st carg becy ceeslklitg wfels biwor c—Tkengeindel desy mathent ghete racouce watingwu

KeyboardInterrupt: 

---
So this model gets better and better at generating text for a while, and then the learning peters out. You can tell because the loss stops going down. Loss is basically how far off the mark the model is. This is a good primer on neural networks if all of this is new: [Neural Networks and Deep Learning](http://neuralnetworksanddeeplearning.com/)

Here is a sample of the final output of this program when I ran it just now:

_“Uped, of anl his of whisthen thistheis thove steresid sed thads. Gont his wishe. Thiter-suen he o coteast bleve, a Gorked oozrerges a coly his spere hest-nrie caml Whaly agrart spewew inting a the a mpo making hacals the theis exprie.”_

This is so far less text-like than the higher n n-gram models. It feels a little different, perhaps because it is doing a different thing. Unlike the n-gram models it does have the capability of capturing finer grained and longer term context. It seems to me to capture the rhythm of text better than the low-n models that are similarly garbled. But it is not language in any useful way. Simple recurrent neural networks are still limited in the amount of context that they can take into account. Since recurrent neural networks were invented, there have been quite a few developments improving on this basic technology.

Stay tuned for more!
